In [1]:
import json
from pathlib import Path
from whoosh import index

In [46]:
from bs4 import BeautifulSoup

In [2]:
sample = "/sbtal/riksdag-video/api_output/H9C120211014fs"

In [17]:
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED, NUMERIC

schema = Schema(videostatus=NUMERIC(stored=True),
                committee=ID(stored=True),
                type=ID(stored=True),
                debatepreamble=ID(stored=True),
                debatetexthtml=ID(stored=True),
                livestreamurl=ID(stored=True),
                activelivespeaker=ID(stored=True),
                id=ID(stored=True),
                dokid=ID(stored=True),
                title=ID(stored=True),
                debatename=ID(stored=True),
                debatedate=ID(stored=True),
                debatetype=ID(stored=True),
                debateurl=ID(stored=True),
                fromchamber=ID(stored=True),
                thumbnailurl=ID(stored=True),
                debateseconds=NUMERIC(stored=True),
                streamurl=ID(stored=True),
                start=NUMERIC(stored=True),
                duration=NUMERIC(stored=True),
                speaker=ID(stored=True),
                party=ID(stored=True),
                subid=ID(stored=True),
                active=ID(stored=True),
                number=NUMERIC(stored=True),
                paragraph=NUMERIC(stored=True),
                text=TEXT(stored=True))

In [54]:
BASE_KEYS = ['videostatus', 'committee', 'type', 'debatepreamble', 'debatetexthtml', 'livestreamurl', 'activelivespeaker', 'id', 'dokid', 'title', 'debatename', 'debatedate', 'debatetype', 'debateurl', 'fromchamber', 'thumbnailurl', 'debateseconds']
def read_api_json(filename):
    infile = str(filename)
    with open(infile) as input:
        data = json.load(input)
    assert "videodata" in data

    if len(data["videodata"]) > 1:
        print(f"More than one 'videodata' in {infile}")

    base = {}
    for key in BASE_KEYS:
        base[key] = data["videodata"][0][key]

    assert "streams" in data["videodata"][0]
    assert "files" in data["videodata"][0]["streams"]
    if len(data["videodata"][0]["streams"]["files"]) > 1:
        print(f"More than one stream: {infile}")
    assert "url" in data["videodata"][0]["streams"]["files"][0]
    base["streamurl"] = data["videodata"][0]["streams"]["files"][0]["url"]

    assert "speakers" in data["videodata"][0]
    speakers = []
    for speaker in data["videodata"][0]["speakers"]:
        cur = {}
        for key in ["start", "duration", "party", "subid", "active", "number"]:
            cur[key] = speaker[key]
        cur["speaker"] = speaker["text"]
        ending = f" ({cur['party']})"
        if cur["speaker"].endswith(ending):
            cur["speaker"] = cur["speaker"][:-len(ending)]
        html = speaker["anftext"]
        soup = BeautifulSoup(html, 'html.parser')
        for para in soup.find_all("p"):
            pg = cur
            pg["text"] = para.text
            speakers.append(pg)
    return base, speakers

In [55]:
a, b = read_api_json(sample)

In [50]:

soup = BeautifulSoup(b[0]["text"], 'html.parser')

In [53]:
for para in soup.find_all("p"):
    print(para.text)

Fru talman! Europa svettas nu på grund av höga elpriser. Det handlar om saker som utsläppsrätter och kol men också om hemmasnickrade problem som i Sverige, med förtida stängd kärnkraft. Det pågår också en strid i Europa och Sverige om kärnkraftens framtid. Tyskland har ju till exempel valt att avveckla och sitter nu i knät på Putin för gasen.
Det diskuteras om kärnkraften ska ses som hållbar eller fasas ut. Tio länder i Europa har gått samman för att värna den svenska och europeiska kärnkraften. Möjligheten lades i regeringens knä, men Sverige skrev inte på ett omhuldat kärnkraftsbrev och tog strid utan valde i stället en avvecklingslinje.
Min fråga, fru talman, är: Varför har Socialdemokraterna och Miljöpartiet valt att inte stå upp för den svenska kärnkraften i EU nu under hösten?


In [18]:
import os, os.path

if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

ix = index.create_in("indexdir", schema)

In [ ]:
API_OUTPUT = Path("/sbtal/riksdag-video/api_output/")
for file in API_OUTPUT.glob("*"):
    print(str(file))